<a href="https://colab.research.google.com/github/Facktorial/ToyDataScraping/blob/main/Orienteering/NabushenostEventu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import requests
import pandas as pd
import io

In [141]:
from enum import Enum

class RaceClass(Enum):
  H = "H"
  D = "D"
  H_OTHER = "H_OTHER"
  D_OTHER = "D_OTHER"
  OTHER = ""

def class_2_gender(r: RaceClass) -> str:
  return "M" if r.value[0] == "H" else "F" if r.value[0] == "D" else ""

def get_gender_akademicky(s: str) -> RaceClass:
  if RaceClass.H.value == s:
    return RaceClass.H
  elif RaceClass.D.value == s:
    return RaceClass.D
  elif "H" in s:
    return RaceClass.H_OTHER
  elif "D" in s:
    return RaceClass.D_OTHER
  return RaceClass.OTHER

In [43]:
# getEventStartLists

In [2]:
def get_csos_club_list():
    url = "https://oris.orientacnisporty.cz/API/"
    params = {
        'format': 'json',
        'method': 'getCSOSClubList'
    }
    response = requests.get(url, params=params)
    response.raise_for_status()  # Check if the request was successful
    return response.json()

In [157]:
# Function to download the CSV file of Ranking and load it into a pandas DataFrame
def download_and_load_csv(gender: str, date: str) -> pd.DataFrame:
    url = "https://oris.orientacnisporty.cz/ranking_export"
    params = {
        'date': date,
        'sport': '1',
        'gender': gender,
        'csv': '1'
    }
    response = requests.get(url, params=params)
    response.raise_for_status()  # Check if the request was successful

    # Load the CSV data into a pandas DataFrame
    csv_data = response.content.decode('utf-8')
    df = pd.read_csv(io.StringIO(csv_data), sep=";")
    return df

In [159]:
# Download and load the CSV file
ranking_dfs = {}
ranking_dfs["M"] = [download_and_load_csv("M", '2024-05-31'), download_and_load_csv("M", '2024-03-31')]
ranking_dfs["F"] = [download_and_load_csv("F", '2024-05-31'), download_and_load_csv("F", '2024-03-31')]
print("Ranking Data:")
_ = [print(x.describe()) for x in ranking_dfs["M"]]
_ = [print(x.describe()) for x in ranking_dfs["F"]]

Ranking Data:
            Poradi          Body        Rank.c   Minule Ran
count  1293.000000   1293.000000   1293.000000  1293.000000
mean    645.800464  26974.600928   3241.496520   616.019335
std     371.444368  23074.497964   2772.804644   380.289796
min       1.000000      0.000000      0.000000     0.000000
25%     324.000000   5540.000000    666.000000   286.000000
50%     647.000000  20744.000000   2493.000000   609.000000
75%     970.000000  47450.000000   5702.000000   947.000000
max    1238.000000  83217.000000  10000.000000  1242.000000
            Poradi          Body        Rank.c   Minule Ran
count  1267.000000   1267.000000   1267.000000  1267.000000
mean    633.169692  26868.119179   3199.081294   551.380426
std     364.537934  22874.767088   2723.605813   367.664606
min       1.000000      0.000000      0.000000     0.000000
25%     317.500000   5619.000000    669.000000   225.500000
50%     634.000000  20669.000000   2461.000000   545.000000
75%     950.500000  47540.

In [167]:
PAST_RANKING = tuple[float, float, float]
RANKING = tuple[float, float, float, PAST_RANKING]


def get_ranking_by_regno(df: pd.DataFrame, minule_df: pd.DataFrame, reg_no: str) -> RANKING:
  filtered_df = df[df['Reg. c.'] == reg_no]
  minule_filtered_df = minule_df[minule_df['Reg. c.'] == reg_no]
  if filtered_df.empty:
    return (float('inf'), -1, -1, (float('inf'), float('inf'), float('inf')))
  if minule_filtered_df.empty:
    minule = filtered_df['Minule Ran'].iloc[0]
    return (
      filtered_df['Poradi'].iloc[0]
      , filtered_df['Body'].iloc[0]
      , filtered_df['Rank.c'].iloc[0]
      , (
          minule if minule > 0 else float('inf')
          , float('inf')
          , float('inf')
      )
    )
  return (
    filtered_df['Poradi'].iloc[0]
    , filtered_df['Body'].iloc[0]
    , filtered_df['Rank.c'].iloc[0]
    , (
        filtered_df['Minule Ran'].iloc[0]
        , minule_filtered_df['Poradi'].iloc[0]
        , minule_filtered_df['Minule Ran'].iloc[0]
    )
  )

In [160]:
get_ranking_by_regno(ranking_dfs["M"][0], ranking_dfs["M"][1], "MOV9500")

(1063, 3931, 472, (0, inf, inf))

In [162]:
get_ranking_by_regno(ranking_dfs["M"][0], ranking_dfs["M"][1], "AOP0201")

(16, 73246, 8802, (52, 52, 94))

In [19]:
def get_event_entries(event_id: int=8657):
    url = "https://oris.orientacnisporty.cz/API/"
    params = {
      'format': 'json',
      'method': 'getEventEntries',
      'eventid': f"{event_id}"
    }
    response = requests.get(url, params=params)
    response.raise_for_status()  # Check if the request was successful
    return response.json()

In [177]:
clubs = get_csos_club_list()
# print("CSOS Club List:")
# print(clubs)

In [178]:
entries = get_event_entries(8657)
# print("Event entries:")
# print(entries)

In [174]:
from dataclasses import dataclass

@dataclass
class Entry:
  classR: RaceClass
  name: str
  regno: int
  ranking: RANKING

  @property
  def ranked(self) -> float:
    if self.ranking is None:
      return None
    return self.ranking[0]

def divide_entries(entries: list[Entry]) -> dict[RaceClass, list[Entry]]:
  ret = {race_class: [] for race_class in RaceClass}
  for entry in entries:
    ret[entry.classR].append(entry)
  for cls, ls in ret.items():
    ret[cls] = sorted(ls, key=lambda entry: entry.ranked)
  return ret

In [175]:
entrs: list[Entry] = []
for x in entries["Data"]:
  obj = entries["Data"][x]
  cls = get_gender_akademicky(obj["ClassDesc"])
  gender = class_2_gender(cls)
  entrs.append(Entry(
      classR=cls
      , name=obj["Name"]
      , regno=obj["RegNo"]
      , ranking=get_ranking_by_regno(ranking_dfs[gender][0], ranking_dfs[gender][1], obj["RegNo"])
       if cls != RaceClass.OTHER else None
  ))

div_entrs = divide_entries(entrs)
# _ = [print(len(x)) for x in div_entrs.values()]
# div_entrs

In [176]:
print("RANKING z ORISU")
for r, ls in div_entrs.items():
  print(r)
  for x in ls:
    print(f"{x.name.ljust(20)} \t{x.ranked}.\t{x.ranking[3]}.")

RANKING z ORISU
RaceClass.H
Vandas Daniel        	5.	(7, 4, 4).
Coufal Jáchym        	8.	(11, 12, 12).
Janovský Tomáš       	10.	(14, 15, 15).
Horvát Petr          	11.	(8, 8, 8).
Metelka Ondřej       	13.	(10, 11, 11).
Kostka Jiří          	16.	(52, 52, 94).
Gajda Jan            	17.	(65, 23, 23).
Adámek Filip         	18.	(19, 19, 19).
Čech Vít             	20.	(23, 30, 36).
Pompura Daniel       	23.	(21, 35, 35).
Hájek Kryštof        	25.	(35, 16, 16).
Škvor Ota            	34.	(22, 22, 21).
Tokár Radim          	62.	(76, 86, 90).
Panovec Kryštof      	75.	(102, 103, 103).
Mareček Šimon        	79.	(63, 67, 66).
Macek Ondřej         	84.	(77, 89, 88).
Thoř Tomáš           	107.	(94, 96, 99).
Pipek Ondřej         	129.	(113, 111, 110).
Měšťan Ondřej        	132.	(131, 139, 158).
Mareš Albert         	133.	(120, 110, 115).
Mokrý Ondřej         	135.	(125, 128, 125).
Čech Petr            	136.	(152, 159, 160).
Pavlovec Dan         	137.	(126, 132, 131).
Mielec Jaromír       	138.	(162,